# 🧪 Testing & Verification Guide

## ✅ Pre-Flight Checklist

Before running the simulation, verify:

### Network Configuration
- [ ] DHCP is **DISABLED** on server
- [ ] All devices have **static IPs** configured
- [ ] All IPs are in **192.168.1.0/24** subnet
- [ ] Server IP: **192.168.1.1**
- [ ] AccessPoint IP: **192.168.1.254**
- [ ] All 9 MCUs have correct IPs (see IP table)

### Physical Connections
- [ ] Server → AccessPoint: Ethernet cable connected
- [ ] All MCUs show **green wireless** connection to AccessPoint
- [ ] All sensors connected to **D0 pin** on respective MCUs
- [ ] No red X marks on any connections

### Code Deployment
- [ ] Server code deployed and saved
- [ ] All 3 Cluster Head codes deployed
- [ ] All 6 Member codes deployed
- [ ] Each code matches the device name
- [ ] All codes saved (green checkmark)

---

## 🚀 System Startup Procedure

### Step 1: Start Server First
1. Click **Server-PT**
2. **Programming** tab
3. Check code is loaded
4. Click **Run** (or it auto-starts)
5. **Wait for**: "MULTI-CLUSTER SERVER STARTED"

**Expected Output:**
```
**************************************************
     MULTI-CLUSTER SERVER STARTED
**************************************************
Listening on: 192.168.1.1:5000
Expecting data from 3 Cluster Heads
Waiting for data...

[Server Heartbeat] Running... (1 min)
```

---

### Step 2: Start All Cluster Heads
Start them **one by one** to see individual startup:

**ClusterHead-1:**
1. Click device
2. **Programming** tab → **Run**
3. **Wait for**: "CLUSTER HEAD 1 STARTED"

**Expected Output:**
```
========================================
CLUSTER HEAD 1 STARTED
========================================
Cluster ID: CH1
Listening on port: 5001
Server: 192.168.1.1:5000
Socket status: True
Waiting for Member1-1 and Member1-2...

[CH1 Status] Active... (1)
```

Repeat for **ClusterHead-2** and **ClusterHead-3**.

---

### Step 3: Start All Member Nodes
Start **all 6 members**. They will immediately begin sending data.

**Member1-1 Expected Output:**
```
Member 1-1 Started [Cluster 1]
Node: M1-1
Target CH: 192.168.1.100:5001
Packet 0 -> M1-1:0
Packet 1 -> M1-1:0
Packet 2 -> M1-1:1
```

---

### Step 4: Verify Data Flow

Watch the console outputs in this order:

**1. Members send data:**
```
[Member1-1] Packet 5 -> M1-1:0
[Member1-2] Packet 5 -> M1-2:1
```

**2. Cluster Heads receive and aggregate:**
```
[ClusterHead-1] Received from 192.168.1.101: M1-1:0
[ClusterHead-1] Received from 192.168.1.102: M1-2:1
[ClusterHead-1] >>> Forwarded to SERVER - Cycle 3
[ClusterHead-1] >>> Data: CH1|M1-1:0|M1-2:1
```

**3. Server receives aggregated data:**
```
==================================================
SERVER RECEIVED DATA!
From: 192.168.1.100:5001 [CLUSTER 1]
Data: CH1|M1-1:0|M1-2:1
Total Packets: 1
CH1: 1 | CH2: 0 | CH3: 0
==================================================
```

---

## 📊 Expected System Behavior

### Normal Operation Timeline (First 10 seconds)

```
T=0s:   Server starts, listening
T=1s:   CH1, CH2, CH3 start, listening
T=2s:   All 6 members start sending
T=4s:   First packets reach Cluster Heads
T=6s:   Cluster Heads aggregate (both members reported)
T=6s:   First aggregated packets reach Server
T=8s:   Second round of member packets sent
T=10s:  Server shows: CH1: 2 | CH2: 2 | CH3: 2
```

### Packet Flow Pattern

```
Every 2 seconds per member:

Member1-1 (2s) ──┐
                  ├──> ClusterHead-1 ──┐
Member1-2 (2s) ──┘                     │
                                       │
Member2-1 (2s) ──┐                     │
                  ├──> ClusterHead-2 ──┼──> Server
Member2-2 (2s) ──┘                     │
                                       │
Member3-1 (2s) ──┐                     │
                  ├──> ClusterHead-3 ──┘
Member3-2 (2s) ──┘
```

---

## 🔧 Troubleshooting Guide

### Problem 1: Server Not Receiving Data

**Symptoms:**
- Server shows "Waiting for data..." forever
- No "SERVER RECEIVED DATA!" messages

**Diagnosis Steps:**

1. **Check Cluster Heads:**
   - Are they showing "Forwarded to SERVER" messages?
   - If NO → Go to Problem 2
   - If YES → Server network issue

2. **Verify Server IP:**
   - Server Config → FastEthernet0
   - Must be: **192.168.1.1**
   - Subnet: **255.255.255.0**

3. **Check Server Code:**
   - Port must be **5000**
   - Look for: `socket.begin(5000)`

4. **Verify Cable:**
   - Server to AccessPoint cable connected?
   - Green lights on both ends?

**Solution:**
- Reconfigure server IP
- Reconnect ethernet cable
- Restart server script

---

### Problem 2: Cluster Head Not Receiving from Members

**Symptoms:**
- CH shows "Waiting for Member..." forever
- No "Received from..." messages

**Diagnosis Steps:**

1. **Check Members:**
   - Are members showing "Packet X sent" messages?
   - If NO → Go to Problem 3
   - If YES → Network routing issue

2. **Verify Member Target IP:**
   - Member code must target correct CH IP
   - Example: Member1-1 → **192.168.1.100** (CH1)
   - Example: Member2-1 → **192.168.1.110** (CH2)

3. **Verify CH Port:**
   - CH1 listens on **5001**
   - CH2 listens on **5002**
   - CH3 listens on **5003**

4. **Check Wireless Connections:**
   - All MCUs show green wireless lines?
   - AccessPoint in range?

**Solution:**
- Fix member target IPs in code
- Move MCUs closer to AccessPoint
- Verify CH listening ports

---

### Problem 3: Member Not Sending Data

**Symptoms:**
- Member console shows no output
- Or shows errors

**Diagnosis Steps:**

1. **Check Sensor Connection:**
   - Sensor connected to MCU?
   - Cable connected to **D0 pin**?

2. **Verify Member IP:**
   - Member has static IP configured?
   - IP matches code comments?

3. **Check Code Deployment:**
   - Correct code for this member?
   - Code saved (green checkmark)?

4. **Look for Errors:**
   - "NameError"? Missing import
   - "AttributeError"? Wrong pin number

**Common Errors & Fixes:**

| Error | Cause | Fix |
|-------|-------|-----|
| `NameError: name 'D0' not defined` | Pin not defined | Use `SENSOR_PIN = 0` |
| `Socket not defined` | Missing import | Add `from udp import *` |
| `No output at all` | Code not running | Click Run button |
| `Connection refused` | Wrong target IP | Check CH IP in code |

**Solution:**
- Verify sensor connection
- Re-check IP configuration
- Redeploy correct code

---

### Problem 4: Only Some Clusters Working

**Symptoms:**
- Server shows: CH1: 5 | CH2: 0 | CH3: 3
- Some clusters not reporting

**Diagnosis:**

1. **Identify Silent Cluster:**
   - Which CH number is 0?
   - Check that Cluster Head

2. **Check That Cluster's Members:**
   - Are both members running?
   - Are they sending to correct CH IP?

3. **Verify CH Listening Port:**
   - Each CH has unique port
   - No port conflicts

**Solution:**
- Start missing member nodes
- Verify target IPs in member code
- Check CH listening port matches

---

### Problem 5: Data Not Aggregating

**Symptoms:**
- CH receives from one member only
- No "Forwarded to SERVER" message

**Cause:**
- CH waiting for BOTH members before forwarding
- Only one member is sending

**Diagnosis:**

1. Check BOTH members of that cluster are running
2. Verify both members target the SAME CH IP and port
3. Look at CH console - does it show "Received from" for both?

**Solution:**
- Start the missing member
- Verify both members have correct CH target

---

## 🎯 Performance Metrics

### Expected Packet Rates

| Device Type | Send Rate | Expected Count (1 min) |
|-------------|-----------|------------------------|
| Each Member | 1 packet / 2 sec | ~30 packets |
| Each CH | 1 aggregate / 4 sec | ~15 packets |
| Server | 3 aggregates / 4 sec | ~45 packets total |

### Server Statistics (After 1 Minute)

```
Total Packets: 45
CH1: 15 | CH2: 15 | CH3: 15
```

If numbers are significantly different, investigate that cluster.

---

## 📸 Visual Verification Using Simulation Mode

### Step 1: Enter Simulation Mode
1. Click **Simulation** button (bottom right)
2. Shows packet-by-packet animation

### Step 2: Filter UDP Packets
1. Click **Edit Filters**
2. Check **UDP** only
3. Uncheck others

### Step 3: Watch Packet Flow
1. Click **Auto Capture / Play**
2. You'll see:
   - **Blue packets**: Member → CH
   - **Green packets**: CH → Server

### Step 4: Inspect Packets
1. Click on a packet in the list
2. Click **Inbound PDU Details** or **Outbound PDU Details**
3. See actual data: "M1-1:0" or "CH1|M1-1:0|M1-2:1"

---

## ✨ Success Indicators

### You know it's working when:

✅ **Server Console Shows:**
```
==================================================
SERVER RECEIVED DATA!
From: 192.168.1.100:5001 [CLUSTER 1]
Data: CH1|M1-1:0|M1-2:1
Total Packets: 15
CH1: 5 | CH2: 5 | CH3: 5
==================================================
```

✅ **All Three Numbers Increasing:**
- CH1: ⬆️
- CH2: ⬆️
- CH3: ⬆️

✅ **No Error Messages Anywhere**

✅ **Packet counts roughly equal** (within 1-2 packets)

✅ **In Simulation Mode**: Blue and green packets flowing smoothly

---

## 🎓 Advanced Testing

### Test 1: Sensor Trigger Test

1. Click on a **Motion Sensor**
2. Move it around (drag)
3. Watch value change from 0 → 1
4. See change propagate through system

### Test 2: Node Failure Test

1. Stop one member node
2. Watch its cluster head:
   - Still receives from other member
   - But doesn't forward (waiting for both)
3. Restart member
4. CH immediately forwards accumulated data

### Test 3: Cluster Head Failure Test

1. Stop one cluster head
2. Its members keep sending (see console)
3. Server stops receiving from that cluster
4. Other clusters unaffected
5. Restart CH → resumes normal operation

### Test 4: Network Stress Test

1. Change member send rate to 0.5 sec (faster)
2. Watch if system keeps up
3. Check for packet loss
4. Monitor server total counts

---

## 📋 Quick Diagnostic Commands

### Check Device Status
```
Device → Programming Tab → Console Output
```

### Verify IP Assignment
```
Device → Config Tab → Interface → IP Address
```

### Test Connectivity
Add to any member code:
```python
print("Testing connectivity...")
print("My IP: " + str(customIP()))
```

### Check Packet Stats
Server shows real-time stats in each received message.

---

## 🆘 Emergency Reset Procedure

If everything is broken:

1. **Stop all scripts** (click Stop on each device)
2. **Close Packet Tracer** (don't save)
3. **Reopen** your .pkt file
4. **Verify IPs** on all devices (Config tab)
5. **Redeploy code** (start with Server, then CHs, then Members)
6. **Start in order**: Server → CHs → Members

---

## 📞 Still Having Issues?

### Common Oversights:

- ❌ DHCP still enabled
- ❌ Wrong template selected (use "UDP Socket - Python")
- ❌ Forgot to save code (green checkmark)
- ❌ Sensor not connected to D0
- ❌ Wireless connections not established
- ❌ Copy-pasted wrong code to device
- ❌ IP mismatch between Config and Code

### Debug Mode

Add this to any device code for more info:
```python
print("DEBUG: " + str(customIP()))
print("DEBUG: Wireless status: " + str(customWireless()))
```

---

## 🎉 Success Criteria

Your project is fully working when:

1. ✅ Server console shows data from all 3 clusters
2. ✅ Packet counts are balanced (CH1≈CH2≈CH3)
3. ✅ No error messages anywhere
4. ✅ In Simulation Mode, packets flow smoothly
5. ✅ Sensor value changes propagate to server
6. ✅ System runs continuously without crashes
7. ✅ All 10 devices (1 server + 9 MCUs) are active